In [2]:
using JuMP, GLPK, LinearAlgebra

### Exercise 2

On a particular day during the tourist season a rent-a-car company must supply cars to four destinations according
to the following schedule:

| Destination | Cars required |
|-------------|---------------|
| A           | 2             |
| B           | 3             |
| C           | 5             |
| D           | 7             |

The company has three branches from which the cars may be supplied. On the day in question, the inventory status
of each of the branches was as follows:

| Branch | Cars available |
|--------|----------------|
| 1      | 6              |
| 2      | 1              |
| 3      | 10             |

The distances between branches and destinations are given by the following table:

|        | Destination |    |   |   |
|--------|:-----------:|----|---|---|
| Branch | A           | B  | C | D |
| 1      | 7           | 11 | 3 | 2 |
| 2      | 1           | 6  | 0 | 1 |
| 3      | 9           | 15 | 8 | 5 |

Plan the day’s activity such that supply requirements are met at a minimum cost (assumed proportional to car-miles
travelled).

Resolução:

Criamos uma matriz $Q$ que representa a quantidade de carros que serão enviados de cada filial a cada destino, onde as linhas representam as filiais e as colunas os destinos. Já $D$ é uma matriz que contém as distâncias entre cada um dos destinos e cada uma das filiais. Queremos minimizar a quilometragem rodada dos carros que é dada pela soma dos elementos da multiplicação elemento por elemento das matrizes $D$ e $Q$. A seguir a formulação LP do problema:

In [3]:
# Modelo e Solver
model2 = Model(GLPK.Optimizer)

#Distâncias entre filiais e os destinos 
D = [7 11 3 2; 1 6 0 1; 9 15 8 5]

#Demanda de carros em cada um dos destinos
de = [2 3 5 7]

#Carros disponíveis em cada filial
c = [6 1 10]

# Variaveis
@variable(model2, Q[1:3, 1:4] >= 0, Int)

# Restrições
@constraint(model2, conD[i = 1:4], sum(Q[:, i]) == de[i])
@constraint(model2, conC[i = 1:3], sum(Q[i, :]) == c[i])

# Função objetivo
@objective(model2, Min, sum(D.*Q))

model2

A JuMP Model
Minimization problem with:
Variables: 12
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 7 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 12 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 12 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: Q, conC, conD

Abaixo o resultado encontrado pelo solver, a solução ótima $= 100$ e a matriz $Q$ com a quantidade de carros .

In [72]:
# Chamada do Solver
optimize!(model2)
 
@show objective_value(model2)
value.(Q)

objective_value(model2) = 100.0


3×4 Array{Float64,2}:
 0.0  1.0  5.0  0.0
 0.0  1.0  0.0  0.0
 2.0  1.0  0.0  7.0

### Exercise 3

The National Association of Securities Dealers Automated Quotation Systems (NASDAQ) is a network system in which quotation information in over-the-counter operations is collected. Users of the system can re-
ceive, in a matter of seconds, buy and sell prices and the exact bid and ask price of each market maker that
deals in a particular security. There are 1700 terminals in 1000 locations in almost 400 cities. The central
processing center is in Trumbull, Conn., with concentration facilities in New York, Atlanta, Chicago, and San
Francisco. On this particular day, the market is quiet, so there are only a few terminals being used. The information they have has to be sent to one of the main processing facilities. The following table gives terminals (supply centers), processing facilities (demand centers), and the time that it takes to transfer a message.

| Terminals       | Trumbull | N.Y. | Atlanta | Chicago | San. Fran. | Supply |
|-----------------|:--------:|------|---------|---------|------------|--------|
| Cleveland       | 6        | 6    | 9       | 4       | 10         | 45     |
| Boston          | 3        | 2    | 7       | 5       | 12         | 90     |
| Houston         | 8        | 7    | 5       | 6       | 4          | 95     |
| Los Angeles     | 11       | 12   | 9       | 5       | 2          | 75     |
| Washinton, D.C. | 4        | 3    | 4       | 5       | 11         | 105    |
| Demand          | 120      | 80   | 50      | 75      | 85         |        |

Essa questão é sobre um mercado de ações estadunidense automatizado. O objetivo é minimizar o tempo que leva para transferir as mensagens de um terminal para as instalações de processamento. Para isso, utiliza-se um racíocinio semelhante ao que foi usado no exercício anterior. $T$ é a matriz com os tempos de transferência e $M$ a matriz com a quantidade de mensagens que cada instalação de processamento vai receber de cada terminal.

item a) Solve, using the minimum matrix method to find an initial feasible solution.

Aqui a itenção é encontrar uma solução viável inicial atráves do metódo de matriz mínimo que pode ser compreendido como um algoritmo guloso que em cada iteração toma a melhor decisão para minimizar o custo. Observe inicialmente a tabela acima e note que o menor tempo é $2$, em qual célula que possui o $2$ há a maior demanda que pode ser suprida pelo fornecedor correspondente? A partir disso escolhe-se a célula $T_{2,2}$. Substituindo o valor antigo por $80$, temos a quantidade de mensagens que deve ser transmitida do terminal correspondente para o centro de demanda. E assim sucessivamente é feito o processo, até que todo a demanda seja suprida. E os elementos na última linha correspondente a demanda sejam iguais à zero.

item b) Are there alternative optimal solutions?

A seguir a formulação LP do problema:

In [22]:
# Modelo e Solver
model3 = Model(GLPK.Optimizer)

T = [6 6 9 4 10; 3 2 7 5 12; 8 7 5 6 4; 11 12 9 5 2; 4 3 4 5 11]

#Demand
D = [120 80 50 75 85]

#Suply
S = [45 90 95 75 105]

# Variaveis
@variable(model3, Q[1:5, 1:5] >= 0, Int)

# Restrições
@constraint(model3, conD[i = 1:5], sum(Q[:, i]) == D[i])
@constraint(model3, conS[i = 1:5], sum(Q[i, :]) == S[i])

# Função objetivo
@objective(model3, Min, sum(T.*Q))

model3

A JuMP Model
Minimization problem with:
Variables: 25
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 10 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 25 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: Q, conD, conS

Sim, existe ao menos uma solução alternativa encontrada que não somente é uma solução viável como também é uma solução ótima. A seguir a solução ótima encontrada pelo solver:

In [7]:
optimize!(model3)
println("Optimal Solutions:")
@show objective_value(model3)
value.(Q)

Optimal Solutions:
objective_value(model3) = 1450.0


5×5 Array{Float64,2}:
  5.0   0.0   0.0  40.0   0.0
 90.0   0.0   0.0   0.0   0.0
  0.0   0.0  50.0  35.0  10.0
  0.0   0.0   0.0   0.0  75.0
 25.0  80.0   0.0   0.0   0.0

### Exercise 4

O objetivo é maximizar o lucro de uma empresa de esportes nas vendas de raquetes de tênis de 5 tipos diferentes compradas de 4 fabricantes. A matriz $P$ contém os preços das raquetes, onde as linhas são as fabricantes $M1, M2, M3$ e $M4$ e as colunas os tipos de raquete. A matriz $Q$ representa a quantidade de raquetes em todos os casos, as colunas são os tipos de raquete e as linhas as fabricantes.

In [17]:
# Modelo e Solver
model4 = Model(GLPK.Optimizer)

P = [5.50 7 8.5 4.5 3; 6 6.5 9 3.5 2; 5 7 9.5 4 2.5; 6.5 5.5 8 5 3.5]

#Qtd máxima de raquetes que cada fabricante pode oferecer
S = [600 500 300 400]

#Qtd de cada tipo de raquete que a empresa deseja contratar
R = [300 200 150 500 400]

# Variaveis
@variable(model4, Q[1:5, 1:4] >= 0, Int)

# Restrições
@constraint(model4, conS[i = 1:4], sum(Q[:, i]) <= S[i])
@constraint(model4, conR[i = 1:5], sum(Q[i, :]) == R[i])

# Função objetivo
@objective(model4, Min, sum(P.*(Q')))

model4

A JuMP Model
Minimization problem with:
Variables: 20
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 20 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 20 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: Q, conR, conS

In [87]:
optimize!(model4)

@show objective_value(model4)
value.(Q)

objective_value(model4) = 6750.0


5×4 Array{Float64,2}:
 300.0    0.0    0.0    0.0
   0.0    0.0    0.0  200.0
   0.0    0.0    0.0  150.0
   0.0  500.0    0.0    0.0
 100.0    0.0  300.0    0.0

### Exercise 15

item a) Existem soluções ótimas que vão além do centro 1 enviar todos os produtos disponíveis para o cliente 1. Para verificar isso, podemos fixar um valor $= 0$ por exemplo para o elemento $Q_{1,1}$ e a partir dele o solver nos dá uma solução que coincide com a solução onde não há restrição para $Q_{1,1}$ e o solver nos devolve que $Q_{1,1} = 300$. Em ambos os casos o custo total é $2700$.

In [38]:
# Modelo e Solver
model15 = Model(GLPK.Optimizer)

C = [-1 3; 1 6; 1 5]

#Avaiable supplies
S = [300 400 900]

#Costumer requirements
R = [800 500]

function exercise15(model, C,S, R)
    # Variaveis
    @variable(model, Q[1:3, 1:2] >= 0, Int)

    # Restrições
    @constraint(model, Q[1,1] == 0)

    @constraint(model, conS[i = 1:3], sum(Q[i, :]) <= S[i])
    @constraint(model, conR[i = 1:2], sum(Q[:, i]) == R[i])

    # Função objetivo
    @objective(model, Min, sum(C.*Q))

    model
end

exercise15(model15,C,S,R)

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: Q, conR, conS

In [47]:
# Chamada do Solver
optimize!(model15)
 
@show objective_value(model15)
value.(Q)

objective_value(model15) = 2700.0


2×3 Array{Float64,2}:
   0.0  400.0  400.0
 300.0    0.0  200.0

item b) Aqui o item pede para assumir que o cliente 2 está localizado numa área onde todas as entregas estão sujeitas a um imposto definido como uma porcentagem do custo de uma unidade do produto. Suponha que a taxa de impostos seja de 10% sobre o custo da unidade, por exemplo:

In [41]:
# Modelo e Solver
model15b = Model(GLPK.Optimizer)

C = [-1 3; 1 6; 1 5]

#Avaiable supplies
S = [300 400 900]

#Costumer requirements
R = [800 500]

# Variaveis
@variable(model15b, Q[1:2, 1:3] >= 0, Int)

# Restrições
@constraint(model15b, Q[1,1] == 0)

@constraint(model15b, conS[i = 1:3], sum(Q[:, i]) <= S[i])
@constraint(model15b, conR[i = 1:2], sum(Q[i, :]) == R[i])

# Função objetivo
@objective(model15b, Min, sum(C.*Q') + sum(C[:,2].*Q'[:,2])*0.1)

model15b

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: Q, conR, conS

In [42]:
optimize!(model15b)
@show objective_value(model15b)
value.(Q)

objective_value(model15b) = 2890.0


2×3 Array{Float64,2}:
   0.0  400.0  400.0
 300.0    0.0  200.0

A resposta é sim, o imposto afeta a solução ótima encontrada no item a)

item c)

In [43]:
# Modelo e Solver
model15c = Model(GLPK.Optimizer)

#Avaiable supplies
Sc = [400 400 900]

exercise15(model15c,C,Sc,R)

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: Q, conR, conS

In [44]:
optimize!(model15c)
@show objective_value(model15c)

value.(Q)

objective_value(model15c) = 2500.0


2×3 Array{Float64,2}:
   0.0  400.0  400.0
 300.0    0.0  200.0

### Exercise 16

After solving a transportation problem with positive shipping costs $c_{ij}$ along all arcs, we increase the supply at
one source and the requirement at one destination in a manner that will maintain equality of total supply and total
demand.

a) Would you expect the shipping cost in the modified problem with a larger total shipment of goods to be higher
than the optimal shipping plan from the original problem?

Depende, se o fornecimento de uma fonte aumentar de modo que um item cujo custo seja baixo fornecido nesse local em comparação com os demais locais, pode ser que essa economia de custos compense o aumento da demanda de outro item e torne a solução ótima desse problema menor do que a original.

b) Solve the following transportation problem:

| Source       | Unit shipping costs to destinations |    |    | Supplies |
|--------------|-------------------------------------|----|----|----------|
|              | D1                                  | D2 | D3 |          |
| S1           | 4                                   | 2  | 4  | 15       |
| S2           | 12                                  | 8  | 4  | 15       |
| Requirements | 10                                  | 10 | 10 |          |

In [7]:
# Modelo e Solver
model16b = Model(GLPK.Optimizer)

#Shipping costs
C = [4 2 4; 12 8 4]

# Variaveis
@variable(model16b, Q[1:2, 1:3] >= 0, Int)

# Restrições
@constraint(model16b, conS[i = 1:2], sum(Q[i, :]) <= 15)
@constraint(model16b, conR[i = 1:3], sum(Q[:, i]) == 10)

# Função objetivo
@objective(model16b, Min, sum(C.*Q))

model16b

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: Q, conR, conS

In [8]:
optimize!(model16b)
@show objective_value(model16b)
value.(Q)

objective_value(model16b) = 130.0


2×3 Array{Float64,2}:
 10.0  5.0   0.0
  0.0  5.0  10.0

c) Increase the supply at source S1 by 1 unit and the demand at demand center D3 by 1 unit, and re-solve the
problem. Has the cost of the optimal shipping plan decreased? Explain this behavior.

In [5]:
# Modelo e Solver
model16b = Model(GLPK.Optimizer)

#Shipping costs
C = [4 2 4; 12 8 4]

S = [16 15]
R = [10 10 11]

# Variaveis
@variable(model16b, Q[1:2, 1:3] >= 0, Int)

# Restrições
@constraint(model16b, conS[i = 1:2], sum(Q[i, :]) <= S[i])
@constraint(model16b, conR[i = 1:3], sum(Q[:, i]) == R[i])

# Função objetivo
@objective(model16b, Min, sum(C.*Q))

model16b

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: Q, conR, conS

In [6]:
optimize!(model16b)
@show objective_value(model16b)
value.(Q)

objective_value(model16b) = 128.0


2×3 Array{Float64,2}:
 10.0  6.0   0.0
  0.0  4.0  11.0

O custo do plano ótimo diminuiu em comparação com o que foi encontrado no item b). O custo de um item a mais demandado em $D3$ é 4 tanto vindo de $S1$ quanto $S2$, em compensação o item a mais no fornecimento de $S1$ troca uma entrega para $D2$ que custava $8$ para $2$. Fazendo com que no final haja uma economia de $2$ e  o resultado ótimo seja $128$.